In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install transformers'

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /private/var/folders/4h/j80shj057l72x50z2f4cgxnw0000gn/T/pip-req-build-zr13na46
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /private/var/folders/4h/j80shj057l72x50z2f4cgxnw0000gn/T/pip-req-build-zr13na46
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit fc6442165255c4ad77fe460f4995a1a3ec83c5eb
  Cloning https://github.com/SKTBrain/KoBERT.git to /private/var/folders/4h/j80shj057l72x50z2f4cgxnw0000gn/T/pip-install-qi76h1i7/kobert-tokenizer_232555044cf647c1bf2ed76ed6e90607
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /private/var/folders/4h/j80shj057l72x50z2f4cgxnw0000gn/T/pip-install-qi76h1i7/kobert-tokenizer_232555044cf647c1bf2ed76ed6e90607
  Resolved https://github.com/SKTBrain/KoBERT.git to commit fc6442165255c4ad77fe460f4995a1a3ec83c5eb
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax e

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np

In [3]:
from tqdm import tqdm, tqdm_notebook

In [4]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
#bertmodel, vocab = get_pytorch_kobert_model('skt/kobert-base-v1',tokenizer.vocab_file)
bertmodel, vocab = get_pytorch_kobert_model()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


using cached model. /Users/nicole/Desktop/nicole/동아리/와이빅타/신입교육세션/프로젝트/.cache/kobert_v1.zip
using cached model. /Users/nicole/Desktop/nicole/동아리/와이빅타/신입교육세션/프로젝트/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
##GPU 사용 시
device = "cuda:0" if torch.cuda.is_available() else "cpu" #torch.device("cuda:0")

In [9]:
import pandas as pd
chatbot_data = pd.read_excel('한국어_단발성_대화_데이터셋 (2).xlsx')

In [10]:
chatbot_data.sample(n=10)

,Sentence,Emotion,Unnamed: 2,Unnamed: 3,Unnamed: 4,슬픔,8128
35617,환자네 가족 날리겠네ㅠ택시기사 진짜 왜저래?!!!!!!!!다음번에 만나면 진짜 확!...,분노,NaN,NaN,NaN,NaN,NaN
24994,머리가 나뻤지만 노력해서 수재가 됐다는 말은 순 구라다~!!,행복,NaN,NaN,NaN,NaN,NaN
8477,자기의 생각되로 답변이 안나온다고 무조건 나쁜인간!,분노,NaN,NaN,NaN,NaN,NaN
22616,솔직히 이런 상황에서 안정적으로 착륙하는 기장님들이 대단하다고 느껴진다..,행복,NaN,NaN,NaN,NaN,NaN
38597,저랑 비슷하네요 참 돈 많고 여유있는 사람의 우울감 고독감?,슬픔,NaN,NaN,NaN,NaN,NaN
28861,남의 나라 가서 너희 나라에서 맛 볼 수 없는 민주주의를 만끽하네.. ㅉㅉㅉ,분노,NaN,NaN,NaN,NaN,NaN
3292,이럴땐 어떡하죠.,걱정,NaN,NaN,NaN,NaN,NaN
26178,닭그네 처단과 더불어 연이은 햄볶한 소식이네요ㅎㅎ,행복,NaN,NaN,NaN,NaN,NaN
10678,기상청 공무원 쓰레기들 모두 파면하고 연금 몰수해라,분노,NaN,NaN,NaN,NaN,NaN
19225,그니깐 12월에 추워지면서 눈온다니깐?,평온,NaN,NaN,NaN,NaN,NaN


In [11]:
chatbot_data.loc[(chatbot_data['Emotion'] == "걱정"), 'Emotion'] = 0
chatbot_data.loc[(chatbot_data['Emotion'] == "분노"), 'Emotion'] = 1
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픔"), 'Emotion'] = 2
chatbot_data.loc[(chatbot_data['Emotion'] == "평온"), 'Emotion'] = 3
chatbot_data.loc[(chatbot_data['Emotion'] == "행복"), 'Emotion'] = 4

In [12]:
data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [13]:
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['언니 동생으로 부르는게 맞는 일인가요..??', '0']
['우리는 미개 합니다', '1']
['지금 내모습은 컴퓨터앞에서 하루종일 취업사이트만 검색중...', '2']
['쉬는 시간에 오답이나 맞추고 있으면 모교로 돌아가서 중간고사 기말고사나 봐야지', '3']
[' 빨리빨리 파악해서 바로 고쳐주니깐 이러니 고객들이 믿고 살수밖에..', '4']
['미안한데 니들은 인간이 어떻게 이런말 쓰지마라', '1']
['모든 사람이 다 저를 싫어하는 것 같아요...', '2']


In [14]:
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)

In [15]:
print(len(dataset_train))
print(len(dataset_test))

31233
10411


In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab,max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab,pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



In [17]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
tok=tokenizer.tokenize

In [19]:
dataset_train[0]

['부끄럽지안소?', '1']

In [20]:
data_train = BERTDataset(dataset_train, 0, 1, tok,vocab,max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok,vocab,max_len, True, False)

In [21]:
data_train[0]

(array([   2, 2423, 5640, 6044, 7318, 6812, 6607,  258,    3,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(9, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [22]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [23]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)

        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [24]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [25]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [26]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss().to(device)

In [27]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [28]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [29]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [30]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/opt/anaconda3/envs/myvenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/489 [00:00<?, ?it/s]

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


epoch 1 batch id 1 loss 1.6305756568908691 train acc 0.28125
epoch 1 batch id 201 loss 0.9537604451179504 train acc 0.4425528606965174
epoch 1 batch id 401 loss 1.03966224193573 train acc 0.5392768079800498
epoch 1 train acc 0.5601035276073619


/opt/anaconda3/envs/myvenv/lib/python3.7/site-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


epoch 3 batch id 1 loss 0.677127480506897 train acc 0.75
epoch 3 batch id 201 loss 0.590668261051178 train acc 0.7833488805970149
epoch 3 batch id 401 loss 0.6074670553207397 train acc 0.8047459476309227
epoch 3 train acc 0.811030163599182


  0%|          | 0/163 [00:00<?, ?it/s]

[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


epoch 3 test acc 0.6980578541874732


In [31]:
torch.save(model, "test1.pt")
#!pwd

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [36]:
KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
tok = tokenizer.tokenize

end = 1

while end == 1 :
    global test_eval
    test_eval=[]
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    data = [sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok,vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()    
    i=0
    while i<5:
      test_eval.append(logits[i])
      i+=1
    tot=0
    min_num=min(test_eval)
    for i in range(len(test_eval)):
      test_eval[i]+=abs(min_num)
      tot+=test_eval[i]
      
    for i in range(len(test_eval)):
      test_eval[i]=round(test_eval[i]/tot*100)
      
    print("걱정"+str(test_eval[0])+"%")
    print("분노"+str(test_eval[1])+"%")
    print("슬픔"+str(test_eval[2])+"%")
    print("평온"+str(test_eval[3])+"%")
    print("행복"+str(test_eval[4])+"%")


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


하고싶은 말을 입력해주세요 : 안녕하세요?


[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


걱정26%
분노0%
슬픔16%
평온26%
행복32%
하고싶은 말을 입력해주세요 : 아 진짜 화난다


[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


...

걱정0%
분노71%
슬픔7%
평온7%
행복14%
하고싶은 말을 입력해주세요 : 반말하지마라 잼민이들아


[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


걱정8%
분노60%
슬픔14%
평온18%
행복0%
하고싶은 말을 입력해주세요 : 


[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:214] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)


걱정35%
분노5%
슬픔9%
평온51%
행복0%
하고싶은 말을 입력해주세요 : 0
